In [1]:
# From documentation at https://github.com/dennisbakhuis/pigeonXT
import pandas as pd
import numpy as np

from pathlib import Path
from pigeonXT import annotate
from IPython.display import display, Image

labels = ["background",'eucalyptus',"tree"]

In [2]:
pd.read_csv("classifier_output.csv",header=None).rename(columns={0:"filename",1:"predicted"})["filename"].to_csv("filenames.csv")

In [3]:
def labelPortion(
    inputFile, 
    labels = ['yes', 'no'], 
    outputFile='output.csv', 
    portionSize=10,
    textColumn='filename',
):
   
    out = Path(outputFile)
    if out.exists():
        outdf = pd.read_csv(out)
        currentId = outdf.index.max() + 1
    else:
        currentId = 0

    indf = pd.read_csv(inputFile)
    examplesInFile = len(indf)
    indf = indf.loc[currentId:currentId + portionSize - 1]
    actualPortionSize = len(indf)
    print(f'{currentId + 1} - {currentId + actualPortionSize} of {examplesInFile}')
    sentences = indf[textColumn].tolist()

    indf["label"]=None

    def updateRow(example, label):
        print(example, label)
        #labs = setLabels([labels.index(y) for y in selectedLabels], len(labels))
        #indf.loc[indf[textColumn] == example, shortLabels] = labs
        indf.loc[indf[textColumn] == example, "label"] = label

    def finalProcessing(annotations):
        if out.exists():
            prevdata = pd.read_csv(out)
            outdata = pd.concat([prevdata, indf]).reset_index(drop=True)
        else:
            outdata = indf.copy()
        outdata.to_csv(out, index=False)
        
    def displayFunction(filename):
        print(filename)
        display(Image("/Users/nagrawal/Documents/SmartPrimer/Smart Primer User Testing Location Photos/" + filename))

    annotated = annotate( 
        sentences, 
        options=labels, 
        buttons_in_a_row=3,
        reset_buttons_after_click=True,
        include_next=False,
        example_process_fn=updateRow,
        final_process_fn=finalProcessing,
        display_fn=displayFunction
    )     
    return indf


In [26]:
annotations = labelPortion('filenames.csv',
                            labels=labels)

501 - 510 of 514


HTML(value='0 of 10 Examples annotated, Current Position: 0 ')

Output()

PXL_20220608_190259592.jpg tree
PXL_20220608_191928690.jpg tree


In [10]:
# peek at current state of output
pd.read_csv("output.csv")

,Unnamed: 0,filename,label
0,0,PXL_20220608_193212390.jpg,eucalyptus
1,1,PXL_20220608_191013698.jpg,eucalyptus
2,2,PXL_20220608_192248412.jpg,background
3,3,PXL_20220608_192133742.jpg,eucalyptus
4,4,PXL_20220608_191249506.jpg,background
...,...,...,...
275,275,PXL_20220608_190713408.jpg,tree
276,276,PXL_20220608_191330469.jpg,tree
277,277,PXL_20220608_190717083.jpg,tree
278,278,PXL_20220608_192816857.jpg,tree


In [99]:
# peek at the classifier outputs by replacing the filename in this cell 
# to get an idea of the errors
df = pd.read_csv("classifier_output.csv",header=None).rename(columns={0:"filename"})
df.loc[df.filename == "PXL_20220608_190704025.jpg"]

,filename,1
219,PXL_20220608_190704025.jpg,eucalyptus
